# HANK estimation including household parameters

This notebook replicates the estimation of the medium-scale HANK model from [Ensemble MCMC Sampling for DSGE Models](https://gregorboehl.com/live/dime_mcmc_boehl.pdf). This is the estimation with the smaller grid, but the estimation of the model with the larger grid is exactly the same. Please refer to the original paper for details.

Let's start with a few imports:

In [1]:
import warnings
import pathos
import emcee
import time

import numpy as np
import pandas as pd
import emcwrap as ew
import grgrlib.hanktools as htools

# make everything reproducible
np.random.seed(0)

The model is sourced out to an external file. Go get it. Its in the ``ressources`` folder if you want to get a closer look.

In [2]:
model = htools.load_model('ressources/hank2_small.py')

Next, let us prepare the data.

In [3]:
# load data and set the correct time indices
d0 = pd.read_csv('ressources/BS_data.csv', sep=';', index_col='date', parse_dates=True).dropna()
d0.index = pd.date_range('1973Q1', periods=len(d0.index), freq='Q')

# select desired time series
series = ['GDP', 'Infl', 'FFR', 'Cons_JPT', 'Lab', 'Inv_JPT', 'Wage']
data = d0[series]['1983Q1':'2008Q4'].to_numpy()

# set measurement errors as a fraction of the standard deviation of the time series
me_sig = np.std(data, axis=0)*1e-1

We will need the model's initial steady state as a reference point.

In [4]:
st = time.time()
hank_ss, ss, unknowns_ss, targets_ss, hank, unknowns, targets, exogenous = model.dag()
print(f"Initial SS took {time.time() - st} seconds.")

# put this in a dictionary for later
jac_info = {'unknowns': unknowns, 'targets': targets, 'exogenous': exogenous, 'T': 300, 'ss': ss}

Initial SS took 5.074892044067383 seconds.


All necessary information for the estimation is expressed in a yaml file. Lets extract everything we need.

In [5]:
# load yaml
est_info = ew.parse_yaml('ressources/hank2.yaml')

# get priors from the yaml
prior = est_info['estimation']['prior']
shocks = est_info['declarations']['shocks']
observables = est_info['declarations']['observables']

# compile priors
frozen_prior, prior_func, bptrans, _, _, prior = htools.get_prior(prior, shocks, verbose=True)

   adding Z_AR_COEF...
   adding rstar_AR_COEF...
   adding G_AR_COEF...
   adding markup_w_AR_COEF...
   adding markup_AR_COEF...
   adding rinv_shock_AR_COEF...
   adding beta_AR_COEF...
   adding Z_SIG_COEF...
   adding rstar_SIG_COEF...
   adding G_SIG_COEF...
   adding markup_w_SIG_COEF...
   adding markup_SIG_COEF...
   adding rinv_shock_SIG_COEF...
   adding beta_SIG_COEF...
Adding parameters to the prior distribution...
   - sig_c as normal with mean 1.5 and std/df 0.375
   - sig_l as normal with mean 2.0 and std/df 0.75
   - chi0 as gamma with mean 0.2 and std/df 0.15
   - tau as beta with mean 0.2 and std/df 0.1
   - sigma_z as normal with mean 1.0 and std/df 0.4
   - phiss as gamma with mean 4.0 and std/df 2.0
   - zeta_p as beta with mean 0.5 and std/df 0.1
   - zeta_w as beta with mean 0.5 and std/df 0.1
   - iota_p as beta with mean 0.5 and std/df 0.15
   - iota_w as beta with mean 0.5 and std/df 0.15
   - phi_pi as gamma with mean 1.5 and std/df 0.25
   - phi_y as gamma 

Next step is to define all relevant functions for the estimation.

In [6]:
def data_func(x, ss, data):
    """Remove intercept from series
    """

    data_adj = np.empty_like(data)
    data_adj[:, 0] = data[:, 0] - ss['ybar']  # y
    data_adj[:, 1] = data[:, 1] - ss['pistar']  # pi
    data_adj[:, 2] = data[:, 2] - ss['rstar']  # i
    data_adj[:, 3] = data[:, 3] - ss['ybar']  # c
    data_adj[:, 4] = data[:, 4] - ss['n_obs']  # n
    data_adj[:, 5] = data[:, 5] - ss['ybar']  # I
    data_adj[:, 6] = data[:, 6] - ss['ybar']  # w

    return data_adj

In [7]:
def ss_func(ss, x):
    """Calculate steady state
    """

    ss['pi'] = ss['pistar']/100
    ss['i'] = ss['rstar']/100
    ss['r'] = (1 + ss['i']) / (1 + ss['pi']) - 1

    # the actual function to calculate the steady state
    ss = hank_ss.solve_steady_state(ss, unknowns_ss, targets_ss, solver="hybr")
    ss = hank.steady_state(ss)

    return ss

In [8]:
def log_ll(x):
    """A single posterior density evaluation
    """ 
    
    x = bptrans(x)
    x = np.array(x)

    # check prior first, and exit already if infinity
    lprior = prior_func(x)
    if np.isinf(lprior):
        return -np.inf

    # calculate likelihood
    llike, ss_local = htools.get_ll(x, hank, data, data_func, me_sig, jac_info, list(prior), observables, shocks, ss_func=lambda ss, x: ss_func(ss, x), debug=False)

    # exit if unsuccessful
    if ss_local is None:
        return -np.inf
    return llike + lprior

Finally, set some parameters. 

In [9]:
# number of chains. This machine has 48 cores.
nchain = 48*4
# number of iterations
nsteps = 2000

# initialize DIME MCMC
move = ew.DIMEMove(aimh_prob=0.1)
# decide where to store the results
backend_name = 'hank_small0.h5'

The last thing we need is a sample from the prior to initialize our chains.

In [ ]:
# disable warnings. Bad likelihood draws are dealt by the log_ll func
warnings.filterwarnings('ignore')
# initialize a parallel pool. I like pathos
pool = pathos.pools.ProcessPool()

# get a prior sample (in parameter space)
p0 = ew.get_prior_sample(frozen_prior, nchain, mapper=pool.uimap, check_func=lambda x: log_ll(bptrans(x, False)), debug=False)
# transform into proposal space
p0pspace = bptrans(p0, False)

  0%|          | 0/192 [00:00<?, ?it/s]

That's it. We can finally start sampling...

In [ ]:
# initialize the storage
backend = emcee.backends.HDFBackend(backend_name)

# sample the sampler
sampler = ew.run_mcmc(log_ll, p0=p0pspace, nsteps=nsteps, moves=move, tune=500, priors=prior, prior_transform=bptrans, backend=backend, update_freq=100, pool=pool, maintenance_interval=10)

# also save some additional infos
ew.save_to_backend(sampler, {'tune': 500, 'priors': list(prior)})